In [3]:
import math
import numpy as np
# Given values
sigma = 0.28
S0 = 37
K = 40
T = 0.75
r = 0.04
q = 0.02
alpha_temp = 0.45
M = 4

P_amer_bin = 5.0455539623

# Computing tau_final and the bounds for x
tau_final = (T * (sigma**2)) / 2
x_left = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) - (3*sigma*math.sqrt(T))
x_right = (math.log(S0/K)) + ((r - q - ((sigma**2)/2))*T) + (3*sigma*math.sqrt(T))

# Finite difference discretization
delta_tau =( tau_final / M)
N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
delta_x = (x_right - x_left) / N
alpha = delta_tau / (delta_x**2)

print(f"tau_final: {tau_final}, x_left: {x_left}, x_right: {x_right}")
print(f"delta_tau: {delta_tau}, N: {N}, delta_x: {delta_x}, alpha: {alpha}")

# Variables for transformation
a = ((r - q) / (sigma**2)) - 0.5
b = (((r - q) / (sigma**2)) + 0.5)**2 + ((2*q)/(sigma**2))

x_compute = math.log(S0/K)

tau_final: 0.029400000000000003, x_left: -0.8198228806486403, x_right: 0.6350997977092168
delta_tau: 0.007350000000000001, N: 11, delta_x: 0.13226569803253246, alpha: 0.4201388888888888


In [4]:

def f(x):
    return K * math.exp(a*x) * max(1 - math.exp(x), 0)

def g_left(tau):
    return K * math.exp((a*x_left) + (b*tau)) * (1 - math.exp(x_left))

def g_right(tau):
    return 0


In [5]:
def forward_subst(L,b):
    
    N = len(L)
    
    x = np.zeros(N)
    
    x[0] = b[0]/L[0][0]
    
    for j in range(1,N):
        row_sum = 0
        for k in range(j):
            row_sum += L[j][k]*x[k]
        x[j] = (b[j] - row_sum)/L[j][j]
    
    return x

In [6]:
def SOR(A , b , x0 , t1 , t2 , tol1 , tol2 , w  ):
    
    N = len(A)
    
    L_A = np.tril(A)
    U_A = np.triu(A)
    
    D_A = np.zeros((N,N))
    
    for i in range(N):
        D_A[i][i] = A[i][i]
        L_A[i][i] = 0
        U_A[i][i] = 0
    
    c = w*forward_subst(D_A + (w*L_A) , b)
    
    x1 = np.full(N,math.inf)
    
    xprev = x0
    x_new = np.full(N,math.inf)
    _iter = 0
    #residual = np.linalg.norm(b - np.dot(A,x0))
    #consec = np.linalg.norm(x1 - x0)
    if t1 and t2:
        while np.linalg.norm(x_new - x0) > tol1 or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t1:
        while np.linalg.norm(x_new - x0) > tol1: #or np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    elif t2:
        while np.linalg.norm(b - np.dot(A,xprev)) > tol2:
            _iter += 1
            x0 = xprev
            x_new = forward_subst(D_A + (w*L_A) , np.dot((1-w)*D_A - (w*U_A), x0)) + c
            xprev = x_new
    print(_iter) 
    return x_new

In [7]:
ee = [[0 for j in range(N+1)] for i in range(M+1)]

for m in range(1 , M+1):
    for i in range(1, N-1 +1):
        t = x_left + (i*delta_x)
        ttau = m*delta_tau
        
        ee[m][i] = K * math.exp(a*t + b*ttau) * max(1 - math.exp(t) , 0)

In [9]:
import pandas as pd
pd.DataFrame(ee)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0
1,0,23.722715,19.682249,15.425602,10.923431,6.143523,1.050490,0.0,0.0,0.0,0.0,0
2,0,23.911842,19.839165,15.548581,11.010517,6.192502,1.058865,0.0,0.0,0.0,0.0,0
3,0,24.102478,19.997331,15.672541,11.098297,6.241871,1.067307,0.0,0.0,0.0,0.0,0
4,0,24.294633,20.156758,15.797489,11.186778,6.291634,1.075816,0.0,0.0,0.0,0.0,0


In [11]:
def crank_nicolson(M):
    
    delta_tau = tau_final / M
    N = math.floor((x_right - x_left) / math.sqrt(delta_tau / alpha_temp))
    delta_x = (x_right - x_left) / N
    alpha = delta_tau / (delta_x ** 2)
    
    print('M =', M)
    print('N =', N)
    print('Delta_tau =', delta_tau)
    print('Delta_x =', delta_x)
    print('Alpha =', alpha)
    
    U = [[0 for j in range(N+1)] for i in range(M+1)]

    # Initial condition
    for j in range(N+1):
        U[0][j] = f(x_left + j*delta_x)
    
    
    # Left Boundary condtion
    for i in range(1 , M+1):
        U[i][0] = g_left(i*delta_tau)
    
    #Right Boundary Condtion
    for i in range(1 , M+1):
        U[i][N] = g_right(i*delta_tau)
        
        
    # Boundary conditions are handled during the implicit step
    # Create the tridiagonal matrix and the solution vector
    A = np.zeros((N-1, N-1))
    B = np.zeros(N-1)

    # Main loop for the Crank-Nicolson scheme
    for i in range(0, M):

        # Set up the tridiagonal system
        for j in range(1, N):
            if j > 1:
                A[j-1, j-2] = -0.5 * alpha
            A[j-1, j-1] = 1 + alpha
            if j < N - 1:
                A[j-1, j] = -0.5 * alpha
            
            B[j-1] = (0.5 * alpha * U[i][j-1]) + ((1 - alpha) * U[i][j]) + (0.5 * alpha * U[i][j+1])
        
        B[0] += (alpha/2)*U[i+1][0]
        B[-1] += (alpha/2)*U[i+1][N]
        
        # Solve the tridiagonal system
        U_next = SOR(A,B, U[i][1:N],True , False, 1e-6 , 1e-6 , 1.2)
        
        # Addition for early exercise 
        
        # Update the solution
        for j in range(1, N):
            U[i+1][j] = max(U_next[j-1],ee[i+1][j])

        # Boundary conditions
        #U[i+1][0] = g_left((i+1) * delta_tau)
        #U[i+1][N] = g_right((i+1) * delta_tau)

    return U

U = crank_nicolson(M)


M = 4
N = 11
Delta_tau = 0.007350000000000001
Delta_x = 0.13226569803253246
Alpha = 0.4201388888888888
12
11
10
10


In [12]:
import pandas as pd
pd.DataFrame(U)

,0,1,2,3,4,5,6,7,8,9,10,11
0,27.355658,23.535083,19.526575,15.303595,10.837033,6.094932,1.042181,0.000000,0.000000,0.000000,0.000000,0.0
1,27.573749,23.722715,19.682249,15.425602,10.923431,6.179170,2.316312,0.508168,0.076889,0.011628,0.001720,0.0
2,27.793578,23.911842,19.839165,15.548581,11.010517,6.545268,3.059136,1.054376,0.272884,0.058377,0.011058,0.0
3,28.015161,24.102478,19.997331,15.672541,11.098297,6.928647,3.625552,1.537417,0.525445,0.148762,0.035155,0.0
4,28.238510,24.294633,20.156758,15.797489,11.260759,7.279259,4.100235,1.965896,0.795094,0.272622,0.076686,0.0


In [13]:
from scipy.stats import norm

def black_scholes_put(S0, K, r, q, T, sigma):
    d1 = (math.log(S0/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S0 * math.exp(-q * T) * norm.cdf(-d1)
    return put_price
#N = 22
def pointwise_convergence(U):
    x_compute = math.log(S0/K)
    print('x_compute = ' , x_compute)
    for i in range(N):
        if (x_left + (i*delta_x) <= x_compute)  and  (x_compute < x_left + (i+1)*delta_x):
            print(i)
            
            xi, xi1 = x_left + (i*delta_x), x_left + ((i+1)*delta_x)
            Si, Si1 = K*math.exp(xi), K*math.exp(xi1)
            print('S_left = ' , Si)
            print('S_right = ' , Si1)
            Vi, Vi1 = math.exp(-a*xi - b*tau_final)*U[M][i], math.exp(-a*xi1 - b*tau_final)*U[M][i+1]
            V_approx = (((Si1 - S0)*Vi) + ((S0 - Si)*Vi1)) / (Si1 - Si)
            V_exact = P_amer_bin
            error_pointwise = abs(V_approx - V_exact) #/ V_exact

            # Interpolation
            u_xcompute = (((xi1 - x_compute)*U[M][i]) + ((x_compute - xi)*U[M][i+1]) )/ (xi1 - xi)
            V_approx2 = math.exp(-a*x_compute - b*tau_final)*u_xcompute
            error_pointwise2 = abs(V_approx2 - V_exact) #/ V_exact

            return error_pointwise, error_pointwise2

error_pointwise, error_pointwise2 = pointwise_convergence(U)


x_compute =  -0.0779615414697118
5
S_left =  34.13711004369773
S_right =  38.96449168014661


In [6]:
black_scholes_put(S0, K, r, q, T, sigma)

5.885852344690981

In [14]:
error_pointwise, error_pointwise2

(0.05540651259078899, 0.033125971326318115)

In [8]:
#X-grid
x = []
S = []
for j in range(N+1):
    t = x_left + j*delta_x
    x.append(t)
    S.append(K*math.exp(t))

In [9]:
S

[15.371029721090817,
 15.455612972383756,
 15.540661665909852,
 15.626178362890963,
 15.712165638642773,
 15.79862608265233,
 15.885562298656035,
 15.972976904718063,
 16.060872533309187,
 16.149251831386056,
 16.23811746047093,
 16.327472096731817,
 16.41731843106304,
 16.507659169166324,
 16.598497031632228,
 16.68983475402211,
 16.781675086950496,
 16.8740207961679,
 16.96687466264415,
 17.06023948265208,
 17.1541180678518,
 17.248513245375335,
 17.343427857911756,
 17.438864763792814,
 17.534826837079002,
 17.631316967646104,
 17.728338061272225,
 17.82589303972531,
 17.92398484085112,
 18.022616418661702,
 18.121790743424356,
 18.22151080175109,
 18.32177959668854,
 18.42260014780843,
 18.523975491298483,
 18.62590868005388,
 18.728402783769173,
 18.831460889030744,
 18.935086099409755,
 19.0392815355556,
 19.144050335289897,
 19.24939565370098,
 19.3553206632389,
 19.46182855381098,
 19.568922532877874,
 19.676605825550148,
 19.784881674685415,
 19.893753340986006,
 20.0032241030

In [10]:
def compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0, threshold=0.00001):
    
    V_approx = [math.exp(-(a * x[i]) - (b * tau_final)) * U[-1][i] for i in range(len(x))]
    V_exact = [black_scholes_put(S[i], K, r, q,T , sigma) for i in range(len(S))]
    
    valid_indices = [i for i, val in enumerate(V_exact) if val > threshold * S0]
    NRMS = len(valid_indices)
    
    print(valid_indices )
    print(V_exact)
    errors = [((V_approx[i] - V_exact[i])**2) /(V_exact[i]**2) for i in valid_indices]
    rms_error = math.sqrt(sum(errors) / NRMS)
    return rms_error



In [11]:
def compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i):
    # Values of option at nodes of interest
    Vi = math.exp(-a * x[i] - b * tau_final) * U[-1][i]
    Vi_plus_1 = math.exp(-a * x[i+1] - b * tau_final) * U[-1][i+1]
    Vi_minus_1 = math.exp(-a * x[i-1] - b * tau_final) * U[-1][i-1]
    Vi_plus_2 = math.exp(-a * x[i+2] - b * tau_final) * U[-1][i+2]
    Vi_delta_t = math.exp(-a * x[i] - b * (tau_final - delta_tau)) * U[-2][i]
    Vi1_delta_t = math.exp(-a * x[i+1] - b * (tau_final - delta_tau)) * U[-2][i+1]
    
    # Delta
    delta_fd = (Vi_plus_1 - Vi) / (S[i+1] - S[i])
    
    # Gamma
    gamma_fd = ((Vi_plus_2 - Vi_plus_1) / (S[i+2] - S[i+1])) - ((Vi - Vi_minus_1) / (S[i] - S[i-1])) 
    gamma_fd /= (S[i+2] + S[i+1])/2 - (S[i] + S[i-1])/2
    
    # Theta
    V_approx_current = (((S[i+1] - S0) * Vi) + ((S0 - S[i]) * Vi_plus_1)) / (S[i+1] - S[i])
    
    V_approx_previous = (((S[i+1] - S0) * Vi_delta_t) + ((S0 - S[i]) * Vi1_delta_t)) / (S[i+1] - S[i])
    
    delta_t = (2 * delta_tau) / (sigma**2)
    theta_fd = (V_approx_previous - V_approx_current) / delta_t

    return delta_fd, gamma_fd, theta_fd


In [12]:
rms_error = compute_rms_error(U, S, a, b, tau_final, K, r, q, sigma, S0)
print(f"RMS Error: {rms_error}")

# Find i such that xi <= xcompute < xi+1
i = (next(idx for idx, val in enumerate(x) if val > x_compute)) - 1
delta_fd, gamma_fd, theta_fd = compute_greeks(U, S, a, b, tau_final, delta_tau, delta_x, i)

print('I = ' , i )

print(f"Delta (Δ): {delta_fd}")
print(f"Gamma (Γ): {gamma_fd}")
print(f"Theta (Θ): {theta_fd}")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [13]:
print("{:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f}".format(error_pointwise,  error_pointwise2,   rms_error, delta_fd, gamma_fd, theta_fd))


0.000035 0.000024 0.122323 -0.557367 0.038764 -1.807457


In [14]:
# Round values to maintain 6 digits after the decimal
error_pointwise = round(error_pointwise, 6)
error_pointwise2 = round(error_pointwise2, 6)
rms_error = round(rms_error, 6)
delta_fd = round(delta_fd, 6)
gamma_fd = round(gamma_fd, 6)
theta_fd = round(theta_fd, 6)

# Create a DataFrame
data = {
    'error_pointwise': [error_pointwise],
    'gap': [''],  # gap
    'error_pointwise2': [error_pointwise2],
    'gap2': [''],  # gap
    'rms_error': [rms_error],
    'gap3': [''],
    'delta_fd': [delta_fd],
    'gamma_fd': [gamma_fd],
    'theta_fd': [theta_fd]
}
df = pd.DataFrame(data)

# Display the DataFrame
display(df)

,error_pointwise,gap,error_pointwise2,gap2,rms_error,gap3,delta_fd,gamma_fd,theta_fd
0,0.000035,,0.000024,,0.122323,,-0.557367,0.038764,-1.807457
